# Clustering Crypto

In [77]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [80]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")

crypto_df = pd.read_csv('crypto_data.csv', index_col=0)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [81]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply']]

In [82]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df.loc[crypto_df['IsTrading'] == True]
crypto_df['IsTrading'].value_counts()

True    1144
Name: IsTrading, dtype: int64

In [83]:
# Keep only cryptocurrencies with a working algorithm
crypto_df['Algorithm'] = crypto_df['Algorithm'].dropna()

In [84]:
# Remove the "IsTrading" column
crypto_df = crypto_df.drop(columns = 'IsTrading', axis=1)

In [85]:
# Remove rows with at least 1 null value
crypto_df = crypto_df.dropna()

In [86]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df.loc[crypto_df['TotalCoinsMined'] > 0]
crypto_df = crypto_df.loc[crypto_df['TotalCoinSupply'] != '0']

In [87]:
# Drop rows where there are 'N/A' text values
crypto_df = crypto_df[~crypto_df.CoinName.str.contains("N/A")]
crypto_df = crypto_df[~crypto_df.Algorithm.str.contains("N/A")]
crypto_df = crypto_df[~crypto_df.ProofType.str.contains("N/A")]
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000


In [88]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
CoinName = crypto_df[['CoinName']].copy()
crypto_df = crypto_df.drop(columns='CoinName', axis = 1)

In [89]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm


In [90]:
# Create dummy variables for text features
x = pd.get_dummies(crypto_df, columns = ['Algorithm','ProofType'])

In [91]:
# Standardize data
standardized = StandardScaler().fit_transform(x)

### Reducing Dimensions Using PCA

In [92]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(standardized)

In [93]:
# Create a DataFrame with the principal components data
crypto_pca = pd.DataFrame(crypto_pca, index = crypto_df.index)
crypto_pca

,0,1,2
42,-0.317149,0.966021,-0.607129
404,-0.300967,0.966071,-0.607450
1337,2.255810,1.612916,-0.703654
BTC,-0.156609,-1.242977,0.204219
LTC,-0.183111,-1.147414,0.023095
DASH,-0.368419,1.209465,-0.564813
ETC,-0.172805,-1.914442,0.425992
ZEC,-0.172242,-1.952853,0.448920
BTS,-0.252435,1.813580,-0.183012
DGB,0.145945,-1.886049,0.413689


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [94]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_pca)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

C:\Users\anduc\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.



:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [95]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(crypto_pca)

# Predict clusters
predictions = model.predict(crypto_pca)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
crypto_pca["class"] = model.labels_
clustered_df = pd.concat([crypto_pca, crypto_df, CoinName], axis = 1, join = 'outer')
clustered_df

,0,1,2,class,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,CoinName
42,-0.317149,0.966021,-0.607129,0,Scrypt,PoW/PoS,4.199995e+01,42,42 Coin
404,-0.300967,0.966071,-0.607450,0,Scrypt,PoW/PoS,1.055185e+09,532000000,404Coin
1337,2.255810,1.612916,-0.703654,0,X13,PoW/PoS,2.927942e+10,314159265359,EliteCoin
BTC,-0.156609,-1.242977,0.204219,1,SHA-256,PoW,1.792718e+07,21000000,Bitcoin
LTC,-0.183111,-1.147414,0.023095,1,Scrypt,PoW,6.303924e+07,84000000,Litecoin
DASH,-0.368419,1.209465,-0.564813,0,X11,PoW/PoS,9.031294e+06,22000000,Dash
ETC,-0.172805,-1.914442,0.425992,1,Ethash,PoW,1.133597e+08,210000000,Ethereum Classic
ZEC,-0.172242,-1.952853,0.448920,1,Equihash,PoW,7.383056e+06,21000000,ZCash
BTS,-0.252435,1.813580,-0.183012,0,SHA-512,PoS,2.741570e+09,3600570502,Bitshares
DGB,0.145945,-1.886049,0.413689,1,Multiple,PoW,1.140622e+10,21000000000,DigiByte


### Visualizing Results

#### 3D-Scatter with Clusters

In [96]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x=0,
    y=1,
    z=2,
    color="CoinName",
    symbol="Algorithm",
    width=800,
)
#fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Table of Tradable Cryptocurrencies

In [97]:
# Table with tradable cryptos
tradable = clustered_df[["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "class"]]
tradable.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "class"], sortable=True, selectable=True)


:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [98]:
# Print the total number of tradable cryptocurrencies
print(clustered_df['CoinName'].value_counts().sum())

497


#### Scatter Plot with Tradable Cryptocurrencies

In [103]:
# Scale data to create the scatter plot
tradable['TotalCoinSupply'] = tradable['TotalCoinSupply'].astype(int)

OverflowError: Python int too large to convert to C long

In [100]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
tradable.hvplot(x="TotalCoinsMined", y="TotalCoinSupply", kind = 'scatter')

:Scatter   [TotalCoinsMined]   (TotalCoinSupply)